In [1]:
!pip install datasets

from datasets import load_dataset

# Load the ImageNet-Sketch dataset (or any other available dataset)
trainset = load_dataset("evanarlian/imagenet_1k_resized_256", split='train', streaming=True)
testset = load_dataset("evanarlian/imagenet_1k_resized_256", split='test', streaming=True)

README.md:   0%|          | 0.00/42.8k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import torch.nn as nn
import torchvision.models as models
import numpy as np


# Define your image transformation pipeline
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load pretrained ResNet model and modify it to act as a feature extractor
resnet50 = models.resnet50(pretrained=True)
resnet50 = nn.Sequential(*list(resnet50.children())[:-1])  # Remove the final classification layer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet50 = resnet50.to(device)
resnet50.eval()

# Use the map function to apply the transformation to the dataset
def transform_images(dataset, num_samples, batch_size, dataset_name):
    features = []
    labels = []
    for i, sample in tqdm(enumerate(dataset), desc=f'Transforming {dataset_name}', total=num_samples):
        if i >= num_samples:
            break
        image = transform(sample['image'])
        with torch.no_grad():
            outputs = resnet50(image.unsqueeze(0).to(device)).squeeze()
            features.append(outputs.cpu())
            labels.append(torch.tensor(sample['label']))
        if (i + 1) % batch_size == 0:
            x = {f'{dataset_name}_features': torch.vstack(features), f'{dataset_name}_labels': torch.hstack(labels)}
            torch.save(x, f'{dataset_name}_{(i + 1) // batch_size}.pth')
            features = []
            labels = []

trainset_transformed = transform_images(trainset, 200000, 20000, 'trainset')
testset_transformed = transform_images(testset, 40000, 20000, 'testset')

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 164MB/s]
Transforming testset: 100%|██████████| 40000/40000 [13:13<00:00, 50.38it/s]
